### 【 동적 웹 페이지 크롤링 + 로그인 】
- Selenium 요소 선택자
    <pre>
    * By.ID                 => id 속성이 login_btn인 요소
    * By.NAME               => name 속성이 userid인 요소
    * By.CLASS_NAME         => class속성이 menu인 요소 
    * By.TAG_NAME           => <input> 태그 이름
    * By.CSS_SELECTOR       => CSS 선택자 
    * By.XPATH              => "//input[@id='user'] XPath 
    * By.LINK_TEXT          => 화면에 표시된 링크 텍스트 전체 일치
    * By.PARTIAL_LINK_TEXT  => 텍스트 일부만 일치
    </pre>

[1] 모듈 로딩 <hr>

In [ ]:
## - 모듈 로딩 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


[2] 크롤링 관련 함수들 <hr>

In [ ]:
## ============================================================
## 함수이름 :  login_hanbit
## 매개변수 :
## 반환결과 :
## ============================================================
def login_hanbit(driver, user_id: str, user_pw: str):
    ## 해당 사이트 열기
    base_url = "https://www.hanbit.co.kr/academy/"
    driver.get(base_url)

    ## 조건이 만족될 때까지, 최대 N초 동안 계속 대기 
    wait = WebDriverWait(driver, 10)

    ## 1) 상단의 "로그인" 링크 클릭 (텍스트 기준)
    ##    조건(EC)이 가능한 상태가 될 때까지 대기
    login_link = wait.until( EC.element_to_be_clickable((By.LINK_TEXT, "로그인")) )
    login_link.click()

    # 2) 로그인 페이지 로딩 대기 & 아이디/비번 입력창 찾기
    id_input = wait.until( EC.presence_of_element_located((By.ID, "id"))  )
    pw_input = driver.find_element(By.ID, "password")      

    # 3) 아이디/비번 입력
    id_input.clear()
    id_input.send_keys(user_id)

    pw_input.clear()
    pw_input.send_keys(user_pw)

    # 4) 로그인 버튼 클릭 (id, class, text 등 실제 구조에 맞게 수정)
    # 예시: <button id="login_btn">로그인</button>
    # login_btn = driver.find_element(By.ID, "login_btn")  # ← 실제 선택자에 맞게 변경
    # body > div.flex.min-h-screen.flex-col.items-center.justify-center.bg-slate-50.p-4.sm\:p-6.md\:p-8 > div.w-full.max-w-\[480px\].rounded-2xl.bg-white\/80.backdrop-blur-sm.px-6.py-6.sm\:px-8.shadow-lg.ring-1.ring-gray-100\/50.transition-all.duration-300 > form > div > button
    # login_btn = wait.until(
    # EC.element_to_be_clickable(
    #     (By.CSS_SELECTOR, "button[data-sentry-element='Button']") )
    # )

    ## -> 방법2: 텍스트로 찾기 (XPath)
    login_btn = wait.until(
    EC.element_to_be_clickable( (By.XPATH, "//button[normalize-space()='로그인']") )
    )
    login_btn.click()

    # 5) 로그인 후 메인/마이페이지가 로딩될 때까지 잠시 대기
    # 예) 상단에 "로그아웃" 텍스트가 뜨는지 확인
    wait.until(
        EC.presence_of_element_located((By.LINK_TEXT, "로그아웃"))
    )
    print(" 로그인 성공한 것으로 보입니다.")

In [ ]:
def go_to_cart(driver):
    wait = WebDriverWait(driver, 10)
    
    # 1) 상단/메뉴에서 "장바구니" 링크 클릭
    cart_link = wait.until(
        EC.element_to_be_clickable((By.LINK_TEXT, "장바구니"))
    )
    cart_link.click()

    # 2) 장바구니 페이지 로딩 대기
    #   - 예: 제목에 "장바구니" 포함
    wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR,"#frm > div.table_area_cart"))
    )
    print(" 장바구니 페이지 진입 완료.")


In [ ]:
def scrape_cart_items(driver):
    wait = WebDriverWait(driver, 10)
    print("GGGGGG")
    # 1) 장바구니 목록 행들 찾기
    rows = wait.until(

        EC.presence_of_all_elements_located(
            (By.CSS_SELECTOR, "#frm > div > table > tbody > tr")  # ← 실제 클래스/구조 확인
        )
    )
    
    print(rows)
    items = []

    for row in rows:
        # 2) 상품명 #frm > div > table > tbody > tr:nth-child(1) > td.left.info > p > a
        try:
            title_el = row.find_element(By.CSS_SELECTOR, "p.txt_nomal>a")
            title = title_el.text.strip()
        except:
            title = ""

        # 3) 가격 #frm > div > table > tbody > tr:nth-child(1) > td:nth-child(4) > p
        try:
            price_el = row.find_element(By.CSS_SELECTOR, ".price")
            price = price_el.text.strip()
        except:
            price = ""

        items.append((title, price))

    print(f"장바구니 상품 {len(items)}개")
    for i, (title, price) in enumerate(items, start=1):
        print(f"{i:02d}. {title} / {price}")

    return items


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# 위에서 만든 함수들 import 했다고 가정:
# login_hanbit, go_to_cart, scrape_cart_items

def main():
    USER_ID = "id"
    USER_PW = "pass"

    options = Options()
    # 디버깅할 땐 창이 보이는 게 좋습니다. 필요하면 headless로 바꿀 수 있어요.
    # options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)

    try:
        login_hanbit(driver, USER_ID, USER_PW)
        go_to_cart(driver)
        items = scrape_cart_items(driver)

        # TODO: 원하시면 여기서 pandas DataFrame 만들고 CSV로 저장할 수도 있음

    finally:
        # 테스트할 땐 잠깐 멈추고 눈으로 확인해도 좋아요
        import time; time.sleep(10)
        driver.quit()

if __name__ == "__main__":
    main()
